# Chapter 15: Introduction to Generics

---

## 15.1 The Problem Generics Solve

Generics enable you to write flexible, reusable code that works with multiple types while maintaining type safety. Without generics, you would have to either sacrifice type safety (using `any`) or write redundant code for each type.

### 15.1.1 Type Safety Without Generics

**The Problem:**

```typescript
// Without generics - using any (loses type safety)
function identityAny(value: any): any {
  return value;
}

const resultAny = identityAny("hello");
// resultAny is any - no autocomplete, no type checking
// resultAny.toFixed(); // No error at compile time, crashes at runtime

// Without generics - using specific types (not reusable)
function identityString(value: string): string {
  return value;
}

function identityNumber(value: number): number {
  return value;
}

function identityBoolean(value: boolean): boolean {
  return value;
}

// Must choose specific function
const str = identityString("hello"); // OK
const num = identityNumber(42);      // OK
// const bad = identityString(42);   // Error - not flexible
```

### 15.1.2 Code Reusability Challenges

**Duplication Without Generics:**

```typescript
// Without generics - duplicate implementations
class StringStack {
  private items: string[] = [];
  
  push(item: string): void {
    this.items.push(item);
  }
  
  pop(): string | undefined {
    return this.items.pop();
  }
  
  peek(): string | undefined {
    return this.items[this.items.length - 1];
  }
}

class NumberStack {
  private items: number[] = [];
  
  push(item: number): void {
    this.items.push(item);
  }
  
  pop(): number | undefined {
    return this.items.pop();
  }
  
  peek(): number | undefined {
    return this.items[this.items.length - 1];
  }
}

// With generics - single implementation
class Stack<T> {
  private items: T[] = [];
  
  push(item: T): void {
    this.items.push(item);
  }
  
  pop(): T | undefined {
    return this.items.pop();
  }
  
  peek(): T | undefined {
    return this.items[this.items.length - 1];
  }
  
  isEmpty(): boolean {
    return this.items.length === 0;
  }
}

// Usage
const stringStack = new Stack<string>();
stringStack.push("hello");
const str = stringStack.pop(); // Type: string | undefined

const numberStack = new Stack<number>();
numberStack.push(42);
const num = numberStack.pop(); // Type: number | undefined

// Type safety maintained
// stringStack.push(42); // ❌ Error: number not assignable to string
// numberStack.push("hello"); // ❌ Error: string not assignable to number
```

---

## 15.2 Generic Functions

Generic functions allow you to write functions that work with any type while preserving type information.

### 15.2.1 Basic Generic Function Syntax

**Declaring Generic Functions:**

```typescript
// Generic function syntax
function identity<T>(value: T): T {
  return value;
}

// The <T> declares a type parameter
// T is a placeholder that will be replaced with an actual type

// TypeScript infers the type from the argument
const result1 = identity("hello"); // T is inferred as string
// result1 is string

const result2 = identity(42); // T is inferred as number
// result2 is number

const result3 = identity({ name: "John" }); // T is inferred as { name: string }
// result3 has autocomplete for 'name'

// Explicit type argument (rarely needed)
const result4 = identity<number>(100); // Explicitly T = number
const result5 = identity<string>("explicit"); // Explicitly T = string

// Multiple statements in generic function
function wrapInArray<T>(value: T): T[] {
  const array: T[] = []; // Can use T inside function
  array.push(value);
  return array;
}

const wrapped = wrapInArray("test"); // string[]

// Generic function with multiple parameters
function pair<T, U>(first: T, second: U): [T, U] {
  return [first, second];
}

const p = pair("hello", 42); // [string, number]
const p2 = pair(1, true); // [number, boolean]
```

**Generic Function Syntax:**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Generic Function Syntax                           │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   function name<T>(param: T): T {                                   │
│     return param;                                                   │
│   }                                                                 │
│                                                                     │
│   Components:                                                       │
│   ┌──────────────────────────────────────────────────────────────┐ │
│   │ function identity<T>(value: T): T {                          │ │
│   │            │      │      │   │                               │ │
│   │            │      │      │   └── Return type uses T          │ │
│   │            │      │      └────── Parameter uses T             │ │
│   │            │      └───────────── Type parameter declaration │ │
│   │            └──────────────────── Function name              │ │
│   └──────────────────────────────────────────────────────────────┘ │
│                                                                     │
│   Type Inference:                                                   │
│   const result = identity("hello");                                   │
│   // TypeScript infers T = string                                    │
│   // result type: string                                             │
│                                                                     │
│   Explicit Type Argument:                                           │
│   const result = identity<string>("hello");                         │
│   // Explicitly specify T = string                                 │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 15.2.2 Type Parameter Inference

TypeScript can often infer generic type parameters from arguments, making explicit type arguments unnecessary.

**How Inference Works:**

```typescript
// TypeScript infers types from arguments
function map<T, U>(array: T[], fn: (item: T) => U): U[] {
  return array.map(fn);
}

// TypeScript infers: T = number, U = string
const strings = map([1, 2, 3], n => n.toString());
// strings is string[]

// TypeScript infers: T = string, U = number
const lengths = map(["a", "bb", "ccc"], s => s.length);
// lengths is number[]

// TypeScript infers: T = { name: string }, U = string
const names = map([{ name: "John" }, { name: "Jane" }], u => u.name);
// names is string[]

// Inference with multiple parameters
function combine<T, U>(a: T, b: U): [T, U] {
  return [a, b];
}

// T = string, U = number
const combined = combine("hello", 42);

// Inference with context
function filter<T>(array: T[], predicate: (item: T) => boolean): T[] {
  return array.filter(predicate);
}

const numbers = [1, 2, 3, 4, 5];
// TypeScript infers T = number from array
const evens = filter(numbers, n => n % 2 === 0);

// When inference fails or is ambiguous
const result = map([1, 2, 3], x => {
  if (x > 2) return "big";
  return null; // TypeScript might infer string | null
});
// Explicit type may be needed: map<number, string | null>(...)
```

### 15.2.3 Explicit Type Arguments

Sometimes you need to explicitly specify generic type parameters when TypeScript can't infer them correctly.

**When to Use Explicit Types:**

```typescript
// When inference is ambiguous
function createPair<T, U>(first: T, second: U): [T, U] {
  return [first, second];
}

// Without explicit types, might not be what you want
const pair1 = createPair("hello", 42); // [string, number] - OK

// But sometimes you need specific types
interface StringNumberPair {
  first: string;
  second: number;
}

// Explicit type arguments
const pair2 = createPair<string, number>("hello", 42);

// When inference picks wrong type
function fetchData<T>(): Promise<T> {
  return fetch("/api").then(r => r.json());
}

// Without explicit type, T is inferred as unknown
// fetchData().then(data => { /* data is unknown */ });

// With explicit type
fetchData<User[]>().then(users => {
  // users is User[]
  users.forEach(u => console.log(u.name));
});

// Multiple type parameters
function convert<T, U>(input: T, converter: (value: T) => U): U {
  return converter(input);
}

// Explicit when inference fails
const result = convert<string, number>(
  "123",
  s => parseInt(s, 10)
);

// Generic constraints with explicit types
interface Comparable<T> {
  compareTo(other: T): number;
}

function sort<T extends Comparable<T>>(items: T[]): T[] {
  return [...items].sort((a, b) => a.compareTo(b));
}

// Must ensure T implements Comparable
class MyItem implements Comparable<MyItem> {
  constructor(public value: number) {}
  compareTo(other: MyItem): number {
    return this.value - other.value;
  }
}

const sorted = sort<MyItem>([new MyItem(3), new MyItem(1)]);
```

---

## 15.3 Generic Constraints

Generic constraints allow you to restrict the types that can be used with a generic parameter, ensuring the type has certain properties or methods.

### 15.3.1 Using `extends` for Constraints

The `extends` keyword in generic constraints specifies that the type parameter must be assignable to a certain type.

**Basic Constraints:**

```typescript
// Constrain T to have a length property
function logLength<T extends { length: number }>(arg: T): T {
  console.log(arg.length);
  return arg;
}

// Works with anything that has length
logLength("hello");     // OK - string has length
logLength([1, 2, 3]);   // OK - array has length
logLength({ length: 10, value: "test" }); // OK - object with length

// logLength(42);         // ❌ Error: number doesn't have length
// logLength({ name: "test" }); // ❌ Error: no length property

// Constrain to specific interface
interface HasId {
  id: number;
}

function findById<T extends HasId>(items: T[], id: number): T | undefined {
  return items.find(item => item.id === id);
}

interface User extends HasId {
  name: string;
}

interface Product extends HasId {
  title: string;
  price: number;
}

const users: User[] = [{ id: 1, name: "John" }];
const products: Product[] = [{ id: 1, title: "Laptop", price: 999 }];

// Works with both types
const user = findById(users, 1);
const product = findById(products, 1);

// Constrain to specific union
type Status = "pending" | "active" | "completed";

function isValidStatus<T extends Status>(status: string): status is T {
  return ["pending", "active", "completed"].includes(status);
}

// Constrain with multiple properties
interface Sortable {
  sort(): void;
  length: number;
}

function sortItems<T extends Sortable>(items: T): T {
  items.sort();
  return items;
}
```

**Constraint Syntax:**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Generic Constraint Syntax                         │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   function name<T extends Constraint>(param: T): T {                  │
│     // T must satisfy Constraint                                     │
│   }                                                                 │
│                                                                     │
│   Constraint Types:                                                   │
│   ┌──────────────────────────────────────────────────────────────┐ │
│   │ // Interface constraint                                      │ │
│   │ T extends { id: number }                                    │ │
│   │                                                                │ │
│   │ // Primitive constraint                                      │ │
│   │ T extends string                                              │ │
│   │                                                                │ │
│   │ // Union constraint                                          │ │
│   │ T extends "a" | "b" | "c"                                     │ │
│   │                                                                │ │
│   │ // Multiple constraints (intersection)                       │ │
│   │ T extends { length: number } & { toString(): string }       │ │
│   │                                                                │ │
│   │ // Class constraint                                            │ │
│   │ T extends Animal                                              │ │
│   └──────────────────────────────────────────────────────────────┘ │
│                                                                     │
│   Multiple Type Parameters with Constraints:                        │
│   function combine<T extends object, U extends object>(             │
│     first: T,                                                        │
│     second: U                                                        │
│   ): T & U {                                                         │
│     return { ...first, ...second };                                  │
│   }                                                                  │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 15.3.2 Constraints with Type Parameters

You can use one type parameter in a constraint for another, creating relationships between types.

**Dependent Type Constraints:**

```typescript
// Constraint using another type parameter
function getProperty<T, K extends keyof T>(obj: T, key: K): T[K] {
  return obj[key];
}

const user = {
  name: "John",
  age: 30,
  email: "john@example.com"
};

// K is constrained to keyof T ("name" | "age" | "email")
const name = getProperty(user, "name");     // Type: string
const age = getProperty(user, "age");       // Type: number
// const invalid = getProperty(user, "phone"); // ❌ Error: "phone" not in keys

// Constraint ensuring array type
function firstElement<T extends any[]>(arr: T): T[0] {
  return arr[0];
}

const first = firstElement(["a", "b", "c"]); // Type: string
const nums = firstElement([1, 2, 3]);       // Type: number

// firstElement("hello"); // ❌ Error: string doesn't extend array

// Constraint ensuring specific method exists
function logLength<T extends { length: number }>(arg: T): T {
  console.log(`Length: ${arg.length}`);
  return arg;
}

// Constraint with default
function createArray<T = string>(length: number, value: T): T[] {
  return Array(length).fill(value);
}

const strings = createArray(3, "x");      // T inferred as string
const numbers = createArray<number>(3, 0); // T explicitly number
```

### 15.3.3 Using Type Parameters in Constraints

**Advanced Constraint Patterns:**

```typescript
// Ensure two parameters have the same keys
function mergeObjects<T extends object, U extends Record<keyof T, any>>(
  obj1: T,
  obj2: U
): T & U {
  return { ...obj1, ...obj2 };
}

// Ensure method returns specific type
interface Serializer<T> {
  serialize(): T;
}

function serialize<T>(obj: Serializer<T>): T {
  return obj.serialize();
}

// Constraint ensuring constructor
type Constructor<T> = new (...args: any[]) => T;

function createInstance<T>(ctor: Constructor<T>): T {
  return new ctor();
}

class User {
  constructor(public name: string) {}
}

const user = createInstance(User); // TypeScript infers T = typeof User

// Complex constraint with mapped types
type EventMap = {
  click: { x: number; y: number };
  submit: { data: FormData };
  error: { message: string };
};

function emitEvent<K extends keyof EventMap>(
  event: K,
  data: EventMap[K]
): void {
  console.log(`Event: ${event}`, data);
}

emitEvent("click", { x: 10, y: 20 }); // OK
emitEvent("submit", { data: new FormData() }); // OK
// emitEvent("click", { data: new FormData() }); // ❌ Error: wrong data shape
```

---

## 15.4 Generic Interfaces

Interfaces can also be generic, allowing you to define reusable interface shapes that work with multiple types.

### 15.4.1 Defining Generic Interfaces

**Generic Interface Syntax:**

```typescript
// Generic interface for a container
interface Container<T> {
  value: T;
  getValue(): T;
  setValue(newValue: T): void;
}

// Implementation with specific type
const stringContainer: Container<string> = {
  value: "hello",
  getValue() {
    return this.value;
  },
  setValue(newValue) {
    this.value = newValue;
  }
};

// Implementation with different type
const numberContainer: Container<number> = {
  value: 42,
  getValue() {
    return this.value;
  },
  setValue(newValue) {
    this.value = newValue;
  }
};

// Generic interface for API responses
interface ApiResponse<T> {
  data: T;
  status: number;
  message: string;
  timestamp: Date;
}

// Usage with different data types
interface User {
  id: number;
  name: string;
}

interface Product {
  sku: string;
  price: number;
}

const userResponse: ApiResponse<User> = {
  data: { id: 1, name: "John" },
  status: 200,
  message: "Success",
  timestamp: new Date()
};

const productResponse: ApiResponse<Product> = {
  data: { sku: "ABC123", price: 99.99 },
  status: 200,
  message: "Success",
  timestamp: new Date()
};

// Generic interface with multiple type parameters
interface Pair<K, V> {
  key: K;
  value: V;
  toString(): string;
}

const stringNumberPair: Pair<string, number> = {
  key: "age",
  value: 30,
  toString() {
    return `${this.key}: ${this.value}`;
  }
};

const numberBooleanPair: Pair<number, boolean> = {
  key: 1,
  value: true,
  toString() {
    return `${this.key}: ${this.value}`;
  }
};
```

### 15.4.2 Implementing Generic Interfaces

**Class Implementation:**

```typescript
// Generic interface
interface Repository<T, ID> {
  findById(id: ID): T | undefined;
  findAll(): T[];
  save(entity: T): void;
  delete(id: ID): boolean;
}

// Implementation for specific types
interface User {
  id: number;
  name: string;
}

class UserRepository implements Repository<User, number> {
  private users: Map<number, User> = new Map();
  
  findById(id: number): User | undefined {
    return this.users.get(id);
  }
  
  findAll(): User[] {
    return Array.from(this.users.values());
  }
  
  save(user: User): void {
    this.users.set(user.id, user);
  }
  
  delete(id: number): boolean {
    return this.users.delete(id);
  }
}

// Generic implementation
class InMemoryRepository<T extends { id: ID }, ID> 
  implements Repository<T, ID> {
  
  private items: Map<ID, T> = new Map();
  
  findById(id: ID): T | undefined {
    return this.items.get(id);
  }
  
  findAll(): T[] {
    return Array.from(this.items.values());
  }
  
  save(entity: T): void {
    this.items.set(entity.id, entity);
  }
  
  delete(id: ID): boolean {
    return this.items.delete(id);
  }
}

interface Product {
  id: string;
  name: string;
  price: number;
}

const productRepo = new InMemoryRepository<Product, string>();
productRepo.save({ id: "1", name: "Widget", price: 9.99 });
const product = productRepo.findById("1"); // Type: Product | undefined
```

---

## 15.5 Generic Classes

Classes can be generic, allowing instances to work with specific types while sharing implementation.

### 15.5.1 Defining Generic Classes

**Generic Class Syntax:**

```typescript
// Generic class for a data structure
class Queue<T> {
  private data: T[] = [];
  
  enqueue(item: T): void {
    this.data.push(item);
  }
  
  dequeue(): T | undefined {
    return this.data.shift();
  }
  
  peek(): T | undefined {
    return this.data[0];
  }
  
  size(): number {
    return this.data.length;
  }
  
  isEmpty(): boolean {
    return this.data.length === 0;
  }
}

// Usage with different types
const stringQueue = new Queue<string>();
stringQueue.enqueue("first");
stringQueue.enqueue("second");
const str = stringQueue.dequeue(); // Type: string | undefined

const numberQueue = new Queue<number>();
numberQueue.enqueue(1);
numberQueue.enqueue(2);
const num = numberQueue.dequeue(); // Type: number | undefined

// Generic class with multiple type parameters
class KeyValuePair<K, V> {
  constructor(
    public key: K,
    public value: V
  ) {}
  
  toString(): string {
    return `${String(this.key)}: ${String(this.value)}`;
  }
  
  getKey(): K {
    return this.key;
  }
  
  getValue(): V {
    return this.value;
  }
}

const pair1 = new KeyValuePair("id", 123);
// Type: KeyValuePair<string, number>

const pair2 = new KeyValuePair(1, true);
// Type: KeyValuePair<number, boolean>

// Generic class with constraints
class SortableArray<T extends number | string> {
  private items: T[] = [];
  
  add(item: T): void {
    this.items.push(item);
  }
  
  sort(): T[] {
    return [...this.items].sort((a, b) => {
      if (a < b) return -1;
      if (a > b) return 1;
      return 0;
    });
  }
}

const numbers = new SortableArray<number>();
numbers.add(3);
numbers.add(1);
numbers.add(2);
console.log(numbers.sort()); // [1, 2, 3]

const strings = new SortableArray<string>();
strings.add("banana");
strings.add("apple");
console.log(strings.sort()); // ["apple", "banana"]

// const mixed = new SortableArray<boolean>(); // ❌ Error: boolean doesn't extend number | string
```

### 15.5.2 Static Members and Generics

Static members of a generic class cannot reference the class's type parameters because they belong to the class constructor, not instances.

**Static Members in Generic Classes:**

```typescript
class GenericClass<T> {
  // Instance members can use T
  instanceValue: T;
  
  constructor(value: T) {
    this.instanceValue = value;
  }
  
  // Instance method uses T
  getValue(): T {
    return this.instanceValue;
  }
  
  // Static members CANNOT use T
  // static staticValue: T; // ❌ Error: Static members cannot reference class type parameters
  
  // Static methods are not generic by default
  static createString(value: string): GenericClass<string> {
    return new GenericClass(value);
  }
  
  // Static method can be generic with its own type parameter
  static create<T>(value: T): GenericClass<T> {
    return new GenericClass(value);
  }
}

// Usage
const instance1 = new GenericClass<number>(42);
const instance2 = GenericClass.createString("hello");
const instance3 = GenericClass.create<boolean>(true);

// Static generic method
class Utility {
  // Generic static method
  static clone<T>(obj: T): T {
    return JSON.parse(JSON.stringify(obj));
  }
  
  // Generic static with constraint
  static compare<T extends { id: number }>(a: T, b: T): boolean {
    return a.id === b.id;
  }
  
  // Generic static factory
  static createArray<T>(length: number, factory: (index: number) => T): T[] {
    return Array.from({ length }, (_, i) => factory(i));
  }
}

const cloned = Utility.clone({ name: "John" });
const areEqual = Utility.compare({ id: 1, name: "A" }, { id: 1, name: "B" });
const numbers = Utility.createArray(5, i => i * 10); // [0, 10, 20, 30, 40]
```

### 15.5.3 Generic Constraints in Classes

**Constraining Class Type Parameters:**

```typescript
// Constraint ensuring T has an id property
interface Identifiable {
  id: string | number;
}

class Repository<T extends Identifiable> {
  private items: Map<string | number, T> = new Map();
  
  add(item: T): void {
    this.items.set(item.id, item);
  }
  
  findById(id: string | number): T | undefined {
    return this.items.get(id);
  }
  
  update(id: string | number, updates: Partial<Omit<T, "id">>): void {
    const existing = this.items.get(id);
    if (existing) {
      Object.assign(existing, updates);
    }
  }
  
  getAll(): T[] {
    return Array.from(this.items.values());
  }
}

// Usage with types that have id
interface User extends Identifiable {
  id: number; // satisfies Identifiable
  name: string;
}

interface Product extends Identifiable {
  id: string; // satisfies Identifiable
  title: string;
  price: number;
}

const userRepo = new Repository<User>();
userRepo.add({ id: 1, name: "John" });
const user = userRepo.findById(1);

const productRepo = new Repository<Product>();
productRepo.add({ id: "prod-1", title: "Widget", price: 9.99 });

// const badRepo = new Repository<{ name: string }>(); 
// ❌ Error: { name: string } doesn't have id property

// Multiple constraints with intersection
interface HasName {
  name: string;
}

interface HasTimestamp {
  createdAt: Date;
}

class AuditableRepository<T extends HasName & HasTimestamp & Identifiable> 
  extends Repository<T> {
  
  audit(item: T): void {
    console.log(`Auditing ${item.name} at ${item.createdAt}`);
    console.log(`ID: ${item.id}`);
  }
}
```

---

## 15.6 Default Type Parameters

Generic type parameters can have default values, making them optional when the type can be inferred or when a sensible default exists.

### 15.6.1 Default Type Parameter Syntax

**Setting Defaults:**

```typescript
// Generic with default type
interface Container<T = string> {
  value: T;
  getValue(): T;
  setValue(value: T): void;
}

// Uses default (string)
const stringContainer: Container = {
  value: "hello",
  getValue() { return this.value; },
  setValue(v) { this.value = v; }
};

// Explicitly specify type
const numberContainer: Container<number> = {
  value: 42,
  getValue() { return this.value; },
  setValue(v) { this.value = v; }
};

// Multiple type parameters with defaults
interface ApiResponse<Data = unknown, Error = string> {
  status: "success" | "error";
  data?: Data;
  error?: Error;
  timestamp: Date;
}

// Use defaults
const response1: ApiResponse = {
  status: "success",
  timestamp: new Date()
};

// Specify first, use default for second
const response2: ApiResponse<User> = {
  status: "success",
  data: { id: 1, name: "John" },
  timestamp: new Date()
};

// Specify both
const response3: ApiResponse<Product, ApiError> = {
  status: "error",
  error: { code: 404, message: "Not found" },
  timestamp: new Date()
};

interface User {
  id: number;
  name: string;
}

interface Product {
  sku: string;
  price: number;
}

interface ApiError {
  code: number;
  message: string;
}
```

### 15.6.2 Ordering of Type Parameters

**Default Parameter Rules:**

```typescript
// Required parameters must come before optional (with defaults)
// ❌ Bad: Required after optional
// interface Bad<T = string, U> { ... }

// ✅ Good: Required first, then optional
interface Good<T, U = string> {
  first: T;
  second: U;
}

// Multiple defaults
interface Multi<A, B = string, C = number> {
  a: A;
  b: B;
  c: C;
}

// Usage
const m1: Multi<boolean> = {
  a: true,
  b: "default",
  c: 0
};

const m2: Multi<boolean, Date> = {
  a: true,
  b: new Date(),
  c: 0
};

const m3: Multi<boolean, Date, bigint> = {
  a: true,
  b: new Date(),
  c: BigInt(100)
};
```

---

## 15.7 Chapter Summary and Exercises

### Chapter Summary

In this chapter, we introduced TypeScript generics:

**Key Takeaways:**

1. **Why Generics**: Enable type-safe reusable code without sacrificing type information or duplicating implementations.

2. **Generic Functions**:
   - Syntax: `function name<T>(param: T): T`
   - Type inference from arguments
   - Explicit type arguments when needed

3. **Generic Interfaces and Classes**:
   - Reusable object shapes
   - Static members cannot use class type parameters
   - Generic constraints ensure type safety

4. **Constraints**:
   - `T extends Constraint` syntax
   - Ensure type has required properties/methods
   - Use `keyof` for property name constraints

5. **Default Type Parameters**:
   - Provide fallback types
   - Make generics optional when sensible
   - Required parameters must come before optional

### Practical Exercises

**Exercise 1: Generic Functions**

Create utility functions:

```typescript
// 1. Create identity<T>(value: T): T that returns the input unchanged

// 2. Create wrapInArray<T>(value: T): T[] that wraps value in array

// 3. Create mapArray<T, U>(array: T[], fn: (item: T) => U): U[] 
//    (reimplement Array.map)

// 4. Create filterArray<T>(array: T[], predicate: (item: T) => boolean): T[]
//    (reimplement Array.filter)

// 5. Create reduceArray<T, U>(array: T[], fn: (acc: U, item: T) => U, initial: U): U
//    (reimplement Array.reduce)

// Test with different types: numbers, strings, objects
```

**Exercise 2: Generic Constraints**

Build a data processing system:

```typescript
// 1. Create an interface HasId { id: string | number }

// 2. Create a generic class DataStore<T extends HasId> with:
//    - private items: T[] = []
//    - add(item: T): void
//    - findById(id: string | number): T | undefined
//    - update(id: string | number, updates: Partial<Omit<T, "id">>): boolean
//    - delete(id: string | number): boolean
//    - getAll(): T[]

// 3. Create interfaces:
//    - Customer extends HasId { name: string; email: string }
//    - Order extends HasId { customerId: string; total: number; items: string[] }

// 4. Create instances:
//    - customerStore = new DataStore<Customer>()
//    - orderStore = new DataStore<Order>()

// 5. Demonstrate that customerStore cannot accept Orders and vice versa
```

**Exercise 3: Generic Interfaces**

Create reusable component interfaces:

```typescript
// 1. Create a generic interface Component<TProps, TState> with:
//    - props: TProps
//    - state: TState
//    - render(): void
//    - setState(newState: Partial<TState>): void

// 2. Create concrete implementations:
//    - ButtonComponent with props: { label: string; onClick: () => void }, state: { clicked: boolean }
//    - InputComponent with props: { placeholder: string }, state: { value: string; focused: boolean }

// 3. Create a generic interface List<TItem> with:
//    - items: TItem[]
//    - add(item: TItem): void
//    - remove(index: number): void
//    - map<U>(fn: (item: TItem) => U): U[]

// 4. Implement List for different item types
```

**Exercise 4: Advanced Generics**

Implement utility types:

```typescript
// 1. Implement your own version of Partial<T>:
//    type MyPartial<T> = { ... }

// 2. Implement your own version of Required<T>:
//    type MyRequired<T> = { ... }

// 3. Implement your own version of Readonly<T>:
//    type MyReadonly<T> = { ... }

// 4. Implement a DeepReadonly<T> that makes all nested properties readonly:
//    type DeepReadonly<T> = { ... }

// 5. Implement a PickByType<T, U> that picks only properties of type U:
//    type PickByType<T, U> = { ... }
//    // Example: PickByType<{ a: string; b: number; c: string }, string>
//    // Should give: { a: string; c: string }

// 6. Test all your implementations with example types
```

**Exercise 5: Generic Constraints**

Build a type-safe event emitter:

```typescript
// 1. Define an EventMap type that maps event names to event data:
//    type EventMap = {
//      click: { x: number; y: number };
//      submit: { formData: FormData };
//      error: { message: string; code: number };
//    }

// 2. Create a generic class EventEmitter<Events extends Record<string, any>> with:
//    - private listeners: Map<keyof Events, Array<(data: any) => void>>
//    - on<K extends keyof Events>(event: K, listener: (data: Events[K]) => void): void
//    - off<K extends keyof Events>(event: K, listener: (data: Events[K]) => void): void
//    - emit<K extends keyof Events>(event: K, data: Events[K]): void

// 3. Create an instance with the EventMap type:
//    const emitter = new EventEmitter<EventMap>();

// 4. Demonstrate type safety:
//    - emitter.on("click", (data) => { data.x }) // data is typed as { x, y }
//    - emitter.emit("click", { x: 10, y: 20 }) // Must provide correct shape
//    - emitter.emit("submit", { x: 10 }) // ❌ Error: wrong event data

// 5. Add a once<K>(event, listener) method that removes listener after first call
```

### Additional Resources

- **TypeScript Handbook - Generics**: https://www.typescriptlang.org/docs/handbook/2/generics.html
- **Generic Constraints**: https://www.typescriptlang.org/docs/handbook/2/generics.html#generic-constraints
- **TypeScript Deep Dive - Generics**: https://basarat.gitbook.io/typescript/type-system/generics
- **Effective TypeScript - Generics**: https://effectivetypescript.com/2020/08/12/generics/

---

## Coming Up Next: Chapter 16 - Generics in Depth

In the next chapter, we will explore:

- Generic interfaces with multiple type parameters
- Generic classes with inheritance
- Generic type aliases
- Generic constraints with `keyof`
- Default type parameters
- Conditional types with generics
- Variance annotations (covariance/contravariance)
- Generic patterns and best practices

Advanced generics unlock the full power of TypeScript's type system, enabling you to build highly reusable, type-safe abstractions for complex scenarios.

---

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../4. advanced_types/14. nullable_types.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='16. generics_in_depth.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
